<a href="https://colab.research.google.com/github/MukulNarwani/CT_Scan_Denoiser/blob/main/DnCnn_notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For this assignment I ended up making a DnCNN (De-noising convolutional neural network) to denoise the dataset. I explain how I arrived to making a DnCNN in a later section.

In [ ]:
from keras.models import Model,Sequential
from tensorflow.keras import  Input
from keras.layers import Flatten, Dense
from tensorflow.keras.applications import VGG16
from keras.optimizers import Adam
from matplotlib import pyplot as plt
import skimage.io as skio
import tensorflow as tf
from tensorflow.python.client import device_lib 
from google.colab import drive
from keras.models import *
from keras.layers import  Input,Conv2D,BatchNormalization,Activation,Lambda,Subtract
from tensorflow.keras.callbacks import ModelCheckpoint, TensorBoard

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


This model uses a deep convolutional network for denoising. It takes in an input of dimensions (512,512,1) and applies a convolution with 64 filters and a kernel size of 3

The next 10 layers are conv layers with 64 filters and a kernel size of 3. They use batch normalization and have a relu activation layer. 

The last conv layer has one filter to recreate the image with a kernel size of 3. 

The model was meant to capture the residual image of the noise, and have that subtracted from the input image. I wanted to divide the input image into quarters of shape (256,256,1), this would have given the model more data to work with. but this was something I didn't get a chance to implement. 

In [ ]:
def DnCNN():
    
    inpt = Input(shape=(512,512,1))
    # First layer
    x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(inpt)
    x = Activation('relu')(x)
    # 10 layers, Conv+BN+relu
    for i in range(10):
        x = Conv2D(filters=64, kernel_size=(3,3), strides=(1,1), padding='same')(x)
        x = BatchNormalization(axis=-1, epsilon=1e-3)(x)
        x = Activation('relu')(x)   
    # last layer, Conv
    x = Conv2D(filters=1, kernel_size=(3,3), strides=(1,1), padding='same')(x)
    x = Subtract()([inpt, x])   # input - noise
    model = Model(inputs=inpt, outputs=x)
    
    return model

In [ ]:
chckpoints = '/content/drive/MyDrive/Colab Notebooks/checkpointsv2/'
checkpoint_callback=ModelCheckpoint(filepath=chckpoints+'weights.{epoch:02d}.hdf5', save_weights_only=True,verbose=1)
tensorboard_callback=TensorBoard()


In [ ]:
import os 
print(os.path.exists('/content/drive/MyDrive/Colab Notebooks/checkpointsv2/'))

True


I import all the tif datasets here and load them into memory. I split the dataset into 400-Training sets and 112-Testing sets, which is ~78%. 

In [ ]:

clean_images = skio.imread("/content/drive/MyDrive/Colab Notebooks/fly_VNC.tif", plugin="tifffile")
noisy_images = skio.imread("/content/drive/MyDrive/Colab Notebooks/fly_VNC_4x_subsampled.tif", plugin="tifffile")

mouse_cortex_noisy=skio.imread("/content/drive/MyDrive/Colab Notebooks/mouse_cortex.tif", plugin="tifffile")
mouse_cortex_clean=skio.imread("/content/drive/MyDrive/Colab Notebooks/mouse_cortex_4x_subsampled.tif", plugin="tifffile")

# For this assignment I created batches of size 5 because I was running into a lot of memory issues.
# And batches of size 5 were the highest size that seemed to worked

training_set = tf.data.Dataset.from_tensor_slices((noisy_images[:400,:,:],clean_images[:400,:,:])).batch(5)
testing_set = tf.data.Dataset.from_tensor_slices((noisy_images[400:,:,:],clean_images[400:,:,:])).batch(5)


In [ ]:
DnCnn = DnCNN()
optimizer = Adam(learning_rate=0.001)
DnCnn.compile(optimizer=optimizer,loss='mean_squared_error',metrics=['accuracy'])
# DnCnn.load_weights('/content/drive/MyDrive/Colab Notebooks/HMS_code_test_xray/checkpoints/weights.25.hdf5')

In [ ]:
history = DnCnn.fit(training_set,epochs = 15, validation_data=testing_set,callbacks=[checkpoint_callback,tensorboard_callback])
plt.plot(history.history['accuracy'],label='accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.show()

In [ ]:
from tifffile import imsave

# There was a weird bug, that wouldn't let me show the images unless 
# I specified the dimensions like below

# You can test the trained network by changing the dimensions 
# to view any slice


pred=secondDnCnn.predict(mouse_cortex_noisy[340:341,:,:])
plt.imshow(pred[0,:,:,0],cmap='gray')
imsave('/content/drive/MyDrive/Colab Notebooks/output.tif',pred)
plt.show()


I trained the network for 15 epochs (which took about 30 minutes), on the 4x_subsampled fly VNC (the noisy VNC image). I passed in a random unseen slice to the trained network and show the output below. The image on the right is the output from the network. 

The network is succesfully learning the noise representation and is denoising the input image, but in doing so also reduces the quality of the image. I suspect this is because of the loss function I used. According to [1] using SSIM for the loss function instead of MSE, would yield a better quality image and improve training times. I would like to rerun this experiment with PSNR and SSIM loss functions. I also suspect reducing the input dimension might improve results as the network would have a smaller dynamic range to focus on. 


<img src ="https://drive.google.com/uc?export=view&id=1V-3ODN0_Nt_YUdelG9q5BlWImrI0V4b4" width="500">          <img src ="https://drive.google.com/uc?export=view&id=1MZArnCJi0wQGmWdI_JogAwTGcyU-P1sp" width="500">


I then ran the network trained on the fly VNC on the mouse cortex, results are consistent across domains, but the drawbacks are worse in the cortex. Since this part of the cortext doesn't have as high of a resolution to begin with, any resolution loss is very detriental to quality. 


<img src ="https://drive.google.com/uc?export=view&id=1-6019Hh5WMa3n71S9TDocanAgnnWzmOT" width="500">

<img src ="https://drive.google.com/uc?export=view&id=1MlasRyiUGgS23vzlKqXykCXUE0qVAOhl" width="500">


When I got the assignment, I read some research papers on networks that would work best for this task. I found that many recent papers were using GAN architectures for denoising. I've been meaning to learn how GANs are implemented so decided to use this as an oppurtunity to learn and experiment with one. 

I was closely following the '*Low-dose CT Image Restoration using generative adversarial networks*' (J. Kim) paper [1]. In the paper they use a DnCNN for the generator and a VGG16 trained as a binary classifier for the discriminator. I wrote the model architecture for both the discrimiantor and the generator but ran into many roadblocks along the way while attempting to train the network. Finally I decided to just use the DnCNN network and train that to denoise the dataset. From what I understand, training a GAN is much more effective at denoising. Another model that I was planning on testing out is '*TomoGAN: low-dose synchrotron x-ray tomography with generative adversarial networks*' (Zhengchun Liu, et. al.). They use an autoencoder style U-Net to model the the noise vector. It seemed like they used a relatively shallow CNN for the discriminator and achieved quite succesful results. 

All the papers that I read used leaky Relu for their activation functions, I am curious to how that would affect the results of my current network. 

If I had more time, I would use this DnCNN and try to get better results by using different loss functions, I'm quite sure that would show the greatest potential for change. Once I have that down I would like to compare how a TomoGan or a normal DcGAN would compete with just the base DnCNN. 

Using PSNR or SSIM seems like the industry standard in determining the quality of the network. Understanding how they work would be crucial in crafting the right loss function for the model. 

I am curious to see how running the network with deeper layers affects the output, and if I were to run this experiment again with a larger dataset I would experiment with constructing deeper/wider CNNs. While training the model, it does not seem like it is prone to overfitting, but using the MSE loss function is unreliable and I don't think it accurately captures the intricacies of this problem.